In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

## Downloading the info

#### USOS_ZONAS_6_7

In [2]:
USOS_ZONA_7 = gpd.read_file("USOS_ZONA_7.shp")
# USOS_ZONA_6 = gpd.read_file("USOS_ZONA_6.shp")

In [5]:
USOS_ZONA_7.columns

Index(['gid', 'id', 'distrito', 'shape_leng', 'shape_area', 'UNuso',
       'geometry'],
      dtype='object')

#### DENUE

In [3]:
schema = "denue"
table = "denue_2022"

manzana = "manzana"
municipio = "municipio"
id = "id"
latitud = "latitud"
longitud = "longitud"
codigo_act = "codigo_act"
nombre_act = "nombre_act"

query_censo = f"""
SELECT 
  "{manzana}",
  "{municipio}",
  "{id}",
  "{latitud}",
  "{longitud}",
  "{codigo_act}",
  "{nombre_act}",
  "geometry"
FROM {schema}.{table}"""

Denue = aup.gdf_from_query(query_censo, geometry_col='geometry')
print(Denue.shape)
Denue.head(2)

(5528698, 8)


,manzana,municipio,id,latitud,longitud,codigo_act,nombre_act,geometry
0,056,Aguascalientes,18016,21.901488,-102.289962,436112,Comercio al por mayor de partes y refacciones ...,POINT (-102.28996 21.90149)
1,030,Jesús María,49318,21.992029,-102.288802,436112,Comercio al por mayor de partes y refacciones ...,POINT (-102.28880 21.99203)


In [5]:
Denue.columns

Index(['manzana', 'municipio', 'id', 'latitud', 'longitud', 'codigo_act',
       'edificio', 'edificio_e', 'nombre_act', 'geometry'],
      dtype='object')

In [6]:
Denue.isna().sum()

manzana             0
municipio           0
id                  0
latitud             0
longitud            0
codigo_act          0
edificio      5094025
edificio_e    5082311
nombre_act          0
geometry            0
dtype: int64

In [7]:
Denue["nombre_act"].unique()

array(['Comercio al por mayor de partes y refacciones nuevas para automóviles, camionetas y camiones',
       'Comercio al por mayor de juguetes y bicicletas',
       'Comercio al por mayor de cemento, tabique y grava',
       'Comercio al por mayor de maquinaria y equipo para otros servicios y para actividades comerciales',
       'Comercio al por mayor de maquinaria y equipo para la industria manufacturera',
       'Comercio al por mayor de maquinaria y equipo para la construcción y la minería',
       'Comercio al por mayor de equipo de telecomunicaciones, fotografía y cinematografía',
       'Comercio al por mayor de equipo y material eléctrico',
       'Comercio al por mayor de pintura',
       'Comercio al por mayor de desechos de papel y de cartón',
       'Comercio al por mayor de electrodomésticos menores y aparatos de línea blanca',
       'Comercio al por mayor de envases en general, papel y cartón para la industria',
       'Comercio al por mayor de materiales metálicos par

In [6]:
# Función para obtener el tipoCenCom más común, manejando vacíos
def tipoCenCom_predominante(x):
    if x.dropna().empty:
        return None  
    else:
        return x.value_counts().idxmax()

# Ahora agrupamos usando la función segura
grupo_manzana = Denue.groupby('manzana')['codigo_act'] \
    .agg(tipoCenCom_predominante) \
    .reset_index()

grupo_manzana.head()

,manzana,codigo_act
0,001,461110
1,002,461110
2,003,461110
3,004,461110
4,005,461110


In [9]:
# Cuántas manzanas tienen al menos un tipoCenCom no nulo
manzanas_con_datos = Denue.dropna(subset=['codigo_act'])['manzana'].nunique()

# Cuántas manzanas en total
total_manzanas = Denue['manzana'].nunique()

print(f"Manzanas con datos de tipoCenCom: {manzanas_con_datos} de {total_manzanas}")

Manzanas con datos de tipoCenCom: 314 de 314


In [7]:
# Ver qué sistema de refrencia tiene cada uno:
print(USOS_ZONA_7.crs) 
print(Denue.crs)

EPSG:32613
epsg:4326


In [4]:
Denue = Denue.to_crs(USOS_ZONA_7.crs) # Reproyectar

#### BUILDINGS

In [5]:
# Definir tamañao del chunk:
chunk_size = 50000

# Cargar y visualizar este pedazo de los datos para edificios:
chunks = pd.read_csv('843_buildings.csv.gz', chunksize=chunk_size)
for chunk in chunks:
    chunk.head()
    print(chunk.columns)
    print(chunk.isna().sum())

Index(['latitude', 'longitude', 'area_in_meters', 'confidence', 'geometry',
       'full_plus_code'],
      dtype='object')
latitude          0
longitude         0
area_in_meters    0
confidence        0
geometry          0
full_plus_code    0
dtype: int64
Index(['latitude', 'longitude', 'area_in_meters', 'confidence', 'geometry',
       'full_plus_code'],
      dtype='object')
latitude          0
longitude         0
area_in_meters    0
confidence        0
geometry          0
full_plus_code    0
dtype: int64
Index(['latitude', 'longitude', 'area_in_meters', 'confidence', 'geometry',
       'full_plus_code'],
      dtype='object')
latitude          0
longitude         0
area_in_meters    0
confidence        0
geometry          0
full_plus_code    0
dtype: int64
Index(['latitude', 'longitude', 'area_in_meters', 'confidence', 'geometry',
       'full_plus_code'],
      dtype='object')
latitude          0
longitude         0
area_in_meters    0
confidence        0
geometry          0
full_

In [ ]:
import momepy
from shapely.geometry import Point

archivo_buildings = '843_buildings.csv.gz'
chunk_size = 1000

# Lista para guardar resultados
result_chunks = []

# 2. Procesar el dataset de buildings en chunks
for chunk in pd.read_csv(archivo_buildings, chunksize=chunk_size):
    
    # 3. Convierte el chunk en GeoDataFrame
    gdf_buildings = gpd.GeoDataFrame(
        chunk,
        geometry=gpd.points_from_xy(chunk.longitude, chunk.latitude),
        crs = 32613
    )

    # 4. Asignación espacial: edificios dentro de zonas (intersección)
    joined = gpd.sjoin(gdf_buildings, USOS_ZONA_7, how='left', predicate='intersects')
    
    # Asegurar que el edificio pertenezca a la **zona más cercana** o **por área**
    # joined['nearest_zone'] = momepy.get_nearest_xxx(gdf_buildings, zonas, return_dist=False) # ¿Cómo puedo usar esto?

    # 5. Guardar el resultado de este chunk
    result_chunks.append(joined)

# 6. Unir todos los chunks en un único GeoDataFrame
buildings_zonas = pd.concat(result_chunks, ignore_index=True)

# 7. Agrupar por zonas y contar edificios por UNuso
conteo_edificios_por_zona = buildings_zonas.groupby('UNuso').size().reset_index(name='num_edificios')

# 8. Revisar el resultado
print(buildings_zonas.shape)
buildings_zonas.head()

print(conteo_edificios_por_zona)